In [9]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
import json

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained("/data1/dxw_data/llm/text2vec-large-chinese")
model = AutoModel.from_pretrained("/data1/dxw_data/llm/text2vec-large-chinese")

# 读取 conversation_history.json 文件
with open('/data1/dxw_data/llm/is_agents/agents/six_hats/task5/conversation_history.json', 'r', encoding='utf-8') as file:
    conversation_data = json.load(file)

# 对每个句子生成 embedding
def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # 取均值作为句子的 embedding
    return embeddings.squeeze().cpu().numpy()

# 对每个 message 进行分句处理并生成 embeddings
def cluster_message(message, num_clusters=3):  # 可以根据需要调整聚类数量
    sentences = message.split('。')  # 使用 '。' 将 message 分成句子，可以根据实际情况调整分句方式
    sentences = [sentence for sentence in sentences if sentence.strip()]  # 去掉空句子
    
    if len(sentences) < num_clusters:  # 如果句子数量少于聚类数目，则直接返回整个 message
        return sentences[0]  # 返回第一句作为代表性句子
    
    # 为每个句子生成 embedding
    embeddings = np.array([generate_embedding(sentence) for sentence in sentences])

    # 聚类分析
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(embeddings)

    # 计算每个聚类中与聚类中心最接近的句子
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    
    # 返回最具代表性的句子
    return sentences[closest[0]]  # 取与聚类中心最近的一句

# 输出每个 message 中最具代表性的一句话
output_file = '/data1/dxw_data/llm/is_agents/agents/six_hats/task5/representative_sentences.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    for i, conversation in enumerate(conversation_data):
        agent = conversation['agent']
        message = conversation['message']
        
        # 对每个 message 进行聚类分析，得到最具代表性的一句话
        representative_sentence = cluster_message(message)
        
        # 输出并保存结果
        output_text = f"Message {i+1} ({agent}): {representative_sentence}"
        print(output_text)
        f.write(output_text + '\n')

print(f"代表性句子已保存到 {output_file}")


/home/dxw/anaconda3/envs/llava/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Message 1 (蓝色思考帽(管理)): 白色思考帽(客观)负责客观陈述事实和数据，红色思考帽(情感)表达情绪和预感，黄色思考帽(积极)表达乐观、满怀希望、建设性的观点，黑色思考帽(批判)提出负面意见


/home/dxw/anaconda3/envs/llava/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Message 2 (红色思考帽(情感)): 综合来看，这些牌都表明你需要面对一些情感上的挑战，但同时也表示未来可能会有一些改变和希望


/home/dxw/anaconda3/envs/llava/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

/home/dxw/anaconda3/envs/llava/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Message 16 (黄色思考帽(积极)): 亲爱的朋友，感谢你的积极反馈


/home/dxw/anaconda3/envs/llava/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Message 17 (黑色思考帽(批判)): 在这个过程中，保持积极的心态、沟通和合作、制定发展规划、心理调适、保持耐心和不断学习是非常重要的
Message 18 (绿色思考帽(创新)): 亲爱的朋友，根据之前的讨论，我认为在应对职业转变的过程中，可以从以下几个方面进行创新思考：

1. 跨领域发展：可以尝试寻找与自己现有领域相关的跨界机会，例如从技术岗位转向产品经理或者从市场岗位转向设计岗位
代表性句子已保存到 /data1/dxw_data/llm/is_agents/agents/six_hats/task5/representative_sentences.txt


/home/dxw/anaconda3/envs/llava/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
